### Import Dependencies

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import time
from scipy import stats


### Load Datasets

In [6]:
#Load excess market returns for last 22 years 
mktreturns = pd.read_csv('ExcessMktReturn.csv')
mktreturns = mktreturns.drop('Date', axis = 1)   

In [7]:
#Load sector data for last 22 years
sectordata = pd.read_csv('SectorData.csv')
sectordata = sectordata.drop('Date', axis = 1)
sectordata = sectordata.drop('SPY', axis = 1)


In [101]:
print(mktreturns.head())

   Mkt-RF
0    0.00
1    1.93
2   -0.07
3    0.16
4    1.14
-0.07


In [9]:
sectordata.head()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,12.881685,14.553748,9.563667,15.818636,25.008554,16.396906,14.028990,18.487051,19.692347
1,13.016976,14.856742,9.704691,16.094658,25.606020,16.793295,13.970206,18.902487,19.776922
2,13.316548,14.778561,9.768794,16.306993,25.508471,16.764292,14.227410,19.017885,20.139355
3,13.200579,14.700367,9.640586,16.391918,25.581635,16.667618,14.153929,18.740925,19.958132
4,13.442181,14.846981,9.743150,16.625483,25.654800,17.034994,14.389091,19.156361,20.393049


In [10]:
#Converting prices into returns
sectorreturns = sectordata.pct_change()

In [11]:
sectorreturns.head()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.010503,0.020819,0.014746,0.017449,0.023890,0.024175,-0.004190,0.022472,0.004295
2,0.023014,-0.005262,0.006605,0.013193,-0.003810,-0.001727,0.018411,0.006105,0.018326
3,-0.008709,-0.005291,-0.013124,0.005208,0.002868,-0.005767,-0.005165,-0.014563,-0.008998
4,0.018302,0.009973,0.010639,0.014249,0.002860,0.022041,0.016615,0.022167,0.021791


In [102]:
#Function to get betas for each sector 

betalist = []
length = len(sectorreturns)
betadf = pd.DataFrame()
def findbetas(sector):
    betalist = []
    for i in range(length - 500):
        X = mktreturns[i:i+500]
        Y = sectorreturns[sector][i:i+500]
        result = sm.OLS(Y,X).fit()
        beta = result.params[0].round(3)
        beta = beta * mktreturns.iloc[i+501]['Mkt-RF']
        betalist.append(beta)
        
    betadf[sector] = betalist
    
    return betadf
        
        
    

In [103]:
#Run function for each sector 
xlbbeta = findbetas('XLB')
xlebeta = findbetas('XLE')
xlfbeta = findbetas('XLF')
xlibeta = findbetas('XLI')
xlkbeta = findbetas('XLK')
xlpbeta = findbetas('XLP')
xlubeta = findbetas('XLU')
xlvbeta = findbetas('XLV')
xlybeta = findbetas('XLY')


In [104]:
expreturns = xlybeta.copy()

In [105]:
print(expreturns.head())

       XLB      XLE      XLF      XLI      XLK      XLP      XLU      XLV  \
0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1  0.00180  0.00120  0.00600  0.00420  0.00900  0.00240  0.00180  0.00480   
2 -0.00489 -0.00326 -0.01467 -0.01141 -0.02445 -0.00652 -0.00489 -0.01304   
3 -0.01077 -0.00718 -0.03590 -0.02513 -0.05385 -0.01436 -0.01077 -0.02872   
4  0.00162  0.00108  0.00540  0.00378  0.00810  0.00216  0.00162  0.00432   

       XLY  
0      NaN  
1  0.00480  
2 -0.01304  
3 -0.02872  
4  0.00432  


In [129]:
#Find total sum of squares
difference = (sectorreturns[:length - 500].subtract(expreturns))
TSS = (difference.pow(2)).sum(axis = 0)

In [130]:
print(TSS)

XLB    1.904538
XLE    2.252979
XLF    3.052615
XLI    1.557271
XLK    2.312815
XLP    0.639619
XLU    0.955233
XLV    1.042224
XLY    1.662836
dtype: float64


In [131]:
#Find residual sum of squares
difference = (sectorreturns.subtract(sectorreturns.mean(axis=0)))
RSS = (difference.pow(2)).sum(axis = 0)
print(RSS)

XLB    1.264724
XLE    1.669682
XLF    2.016815
XLI    0.991338
XLK    1.445975
XLP    0.517829
XLU    0.801708
XLV    0.722534
XLY    1.062803
dtype: float64


In [132]:
#Find rsquared 
rsq = 1 - (RSS.divide(TSS, axis = 0))
print(rsq)

XLB    0.335942
XLE    0.258900
XLF    0.339316
XLI    0.363413
XLK    0.374799
XLP    0.190410
XLU    0.160720
XLV    0.306738
XLY    0.360849
dtype: float64
